In [3]:
#Define local working drive
import os
os.chdir("/Users/jacobboysen/myprojects/us-fatal-police-encounters/")

In [4]:
#load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #data visualization
import sys

#supress warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [5]:
#Import data
census_data = pd.DataFrame(pd.read_csv('data/census_data_2017.csv')) #2017 U.S. County Census Data
city_data = pd.DataFrame(pd.read_csv('data/uscities.csv')) #US Cities with county
top_city = pd.DataFrame(pd.read_csv('data/city_population_data.csv')) #Top 200 U.S. cities with 2020 population and density
police_data = pd.DataFrame(pd.read_csv('data/police_departments_2018.csv')) #2018 report of the number of active police officers by city

In [4]:
############ DATA PREP ################

#Step 1 - create city_state unique ID to join the top_city and city_data.
'''This step is necessary to avoid issues in joining data due to more than one city with the same name. The city_data 
is joined with the top_city data to bring in the COUNTY value for each city'''
top_city["city_state"] = top_city['Name'] + "_" +  top_city['State']
city_data['city_state'] = city_data['city'] + "_" +  city_data['state_name']

#Step 2 - join the top_city data with the city_data on the city_state unique value
city_df = pd.merge(top_city, city_data, on='city_state', how='left' )
city_df.drop(columns=['city','county_name_all','county_fips_all','population', 
                      'density', 'source', 'timezone', 'ranking', 'zips', 'id','military','incorporated'])

#Step 3 - create county_state unique Id to join the new city_df data with the 2017 Census data by county.
city_df['county_state'] = city_df['county_name'] + "_" +  city_df['State'] #create county_state unique id
census_data['county_state'] = census_data['County'] + "_" +  census_data['State'] #create county_state unique id
city_df = pd.merge(city_df,census_data, on='county_state', how='left' ) #join city_df with census_data

city_df = city_df[['Rank', 'Name', 'county_name','city_state','State_x', '2020_Pop', '2010_Census', 'Change',
       'Density_km', 'Area_km','Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific','IncomePerCap',
       'Poverty', 'Unemployment']] #subset city_df data to include the fields we'll use in our analysis

#Step 4 - create estimated annual population based on known 2010 and 2020 population
city_df["pop_change"] = city_df["2020_Pop"] - city_df["2010_Census"] #calc the 2010 - 2020 population change
city_df["pop_change_per_year"] = city_df["pop_change"] / 10 #divide the population change by 10 to get the estimated annual increase

#Create new annual population estimates based on the known 2010 and 2020 population.
'''This loop creates a new column for each year and uses the pop_change_per_year field to estimate the 
population for given year '''

for i in range(1,10):
     x=2010+i   
     city_df['est_pop_{}'.format(x)] = city_df['2010_Census'] + i * city_df['pop_change_per_year']


In [5]:
#Merge police data with city_df
police_data["city_state"] = police_data["City"] +"_"+ police_data["State"]
city_df = pd.merge(city_df, police_data, on='city_state', how='left' )

In [6]:
#remove duplicate then rename columns
city_df = city_df.drop(columns=['State'])

city_df = city_df.rename(columns={'population': '2018 total police force', 
                                                    'total_officers': '2018 police officers',
                                                    'total_civilians': '2018 police civilians',
                                                    'State_x' : 'state'
                                                   })

In [7]:
#Write city_df to working drive for Step 3 Analysis
city_df.to_csv('data/clean_city_data.csv')